##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Advanced Automatic Differentiation 1

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
</td>
</table>

The [automatic differentiation guide](autodiff.ipynb) includes everything required to calculate gradients. This guide focuses on deeper, less common features of the `tf.GradientTape` api.

## Setup 2

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6

## Controlling gradient recording

In the [automatic differentiation guide](autodiff.ipynb) you saw how to control which variables and tensors are watched by the tape while building the gradient calculation.

The tape also has methods to manipulate the recording.

If you wish to stop recording gradients, you can use `GradientTape.stop_recording()` to temporarily suspend recording.

This may be useful to reduce overhead if you do not wish to differentiate a complicated operation in the middle of your model.  This could include calculating a metric or an intermediate result:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  x_sq = x * x
  with t.stop_recording():
    y_sq = y * y
  z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

If you wish to start over entirely, use `reset()`.  Simply exiting the gradient tape block and restarting is usually easier to read, but you can use `reset` when exiting the tape block is difficult or impossible.

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Stop gradient 3

In contrast to the global tape controls above, the `tf.stop_gradient` function is much more precise. It can be used to stop gradients from flowing along a particular path, without needing access to the tape itself:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Custom gradients

In some cases, you may want to control exactly how gradients are calculated rather than using the default.  These situations include:

- Não há gradiente definido para uma nova operação que você está escrevendo.
- Os cálculos padrão são numericamente instáveis.
- Você deseja armazenar em cache um cálculo caro do passe direto.
- Você deseja modificar um valor (por exemplo, usando: `tf.clip_by_value` , `tf.math.round` ) sem modificar o gradiente.

Para escrever uma nova operação, você pode usar `tf.RegisterGradient` para configurar a sua própria. Consulte essa página para obter detalhes. (Observe que o registro de gradiente é global, portanto altere-o com cuidado.)

Para os últimos três casos, você pode usar `tf.custom_gradient` .


Aqui está um exemplo que aplica `tf.clip_by_norm` ao gradiente intermediário.

In [ ]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


Consulte o decorador `tf.custom_gradient` para obter mais detalhes.

## Multiple tapes

Multiple tapes interact seamlessly. For example, here each tape watches a different set of tensors:

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
  tape0.watch(x0)
  tape1.watch(x1)

  y0 = tf.math.sin(x0)
  y1 = tf.nn.sigmoid(x1)

  y = y0 + y1

  ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0).numpy()   # cos(x) => 1.0

In [ ]:
tape1.gradient(ys, x1).numpy()   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25


### Higher-order gradients

Operations inside of the `GradientTape` context manager are recorded for automatic differentiation. If gradients are computed in that context, then the gradient computation is recorded as well. As a result, the exact same API works for higher-order gradients as well. For example:

In [ ]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    y = x * x * x

  # Compute the gradient inside the outer `t2` context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

While that does give you the second derivative of a *scalar* function, this pattern does not generalize to produce a Hessian matrix, since `GradientTape.gradient` only computes the gradient of a scalar. To construct a Hessian, see the [Hessian example](#hessian) under the [Jacobian section](#jacobians).

"Nested calls to `GradientTape.gradient`" is a good pattern when you are calculating a scalar from a gradient, and then the resulting scalar acts as a source for a second gradient calculation, as in the following example.


#### Exemplo: regularização de gradiente de entrada

Many models are susceptible to "adversarial examples". This collection of techniques modifies the model's input to confuse the model's output. The [simplest implementation](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) takes a single step along the gradient of the output with respect to the input; the "input gradient".

One technique to increase robustness to adversarial examples is [input gradient regularization](https://arxiv.org/abs/1905.11468), which attempts to minimize the magnitude of the input gradient. If the input gradient is small, then the change in the output should be small too.

Below is a naive implementation of input gradient regularization. The implementation is:

1. Calculate the gradient of the output with respect to the input using an inner tape.
2. Calculate the magnitude of that input gradient.
3. Calculate the gradient of that magnitude with respect to the model.

In [ ]:
x = tf.random.normal([7, 5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
  # The inner tape only takes the gradient with respect to the input,
  # not the variables.
  with tf.GradientTape(watch_accessed_variables=False) as t1:
    t1.watch(x)
    y = layer(x)
    out = tf.reduce_sum(layer(x)**2)
  # 1. Calculate the input gradient.
  g1 = t1.gradient(out, x)
  # 2. Calculate the magnitude of the input gradient.
  g1_mag = tf.norm(g1)

# 3. Calculate the gradient of the magnitude with respect to the model.
dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var.shape for var in dg1_mag]

## Jacobianos


Todos os exemplos anteriores consideraram os gradientes de um alvo escalar em relação a algum(s) tensor(es) de origem.

A [matriz Jacobiana](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) representa os gradientes de uma função com valor vetorial. Cada linha contém o gradiente de um dos elementos do vetor.

O método `GradientTape.jacobian` permite calcular com eficiência uma matriz Jacobiana.

Observe que:

- Como `gradient` : o argumento `sources` pode ser um tensor ou um contêiner de tensores.
- Ao contrário `gradient` : o tensor `target` deve ser um único tensor.

### Fonte escalar

Como primeiro exemplo, aqui está o Jacobiano de um alvo vetorial em relação a uma fonte escalar.

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
  y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

Quando você considera o Jacobiano em relação a um escalar, o resultado tem a forma do **alvo** e fornece o gradiente de cada elemento em relação à fonte:

In [ ]:
print(y.shape)
print(dy_dx.shape)

In [ ]:
plt.plot(x.numpy(), y, label='y')
plt.plot(x.numpy(), dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

### Fonte tensora

Quer a entrada seja escalar ou tensor, `GradientTape.jacobian` calcula com eficiência o gradiente de cada elemento da fonte em relação a cada elemento do(s) destino(s).

Por exemplo, a saída desta camada tem o formato `(10, 7)` :

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
  y = layer(x)

y.shape

E a forma do kernel da camada é `(5, 10)` :

In [ ]:
layer.kernel.shape

A forma do Jacobiano da saída em relação ao kernel são essas duas formas concatenadas:

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

Se você somar as dimensões do alvo, ficará com o gradiente da soma que teria sido calculado por `GradientTape.gradient` :

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape:', g.shape)

j_sum = tf.reduce_sum(j, axis=[0, 1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()
assert delta < 1e-3
print('delta:', delta)

<a id="hessian"> </a>

#### Exemplo: Hessiano

Embora `tf.GradientTape` não forneça um método explícito para construir uma matriz Hessiana, é possível construir uma usando o método `GradientTape.jacobian` .

Nota: A matriz Hessiana contém `N**2` parâmetros. Por esta e outras razões não é prático para a maioria dos modelos. Este exemplo é incluído mais como uma demonstração de como usar o método `GradientTape.jacobian` e não é um endosso à otimização direta baseada em Hessian. Um produto vetorial Hessiano pode ser [calculado eficientemente com fitas aninhadas](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py) e é uma abordagem muito mais eficiente para otimização de segunda ordem.


In [ ]:
x = tf.random.normal([7, 5])
layer1 = tf.keras.layers.Dense(8, activation=tf.nn.relu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.relu)

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    x = layer1(x)
    x = layer2(x)
    loss = tf.reduce_mean(x**2)

  g = t1.gradient(loss, layer1.kernel)

h = t2.jacobian(g, layer1.kernel)

In [ ]:
print(f'layer.kernel.shape: {layer1.kernel.shape}')
print(f'h.shape: {h.shape}')

Para usar este Hessian para uma etapa do método de Newton, você primeiro achataria seus eixos em uma matriz e achataria o gradiente em um vetor:

In [ ]:
n_params = tf.reduce_prod(layer1.kernel.shape)

g_vec = tf.reshape(g, [n_params, 1])
h_mat = tf.reshape(h, [n_params, n_params])

A matriz Hessiana deve ser simétrica:

In [ ]:
def imshow_zero_center(image, **kwargs):
  lim = tf.reduce_max(abs(image))
  plt.imshow(image, vmin=-lim, vmax=lim, cmap='seismic', **kwargs)
  plt.colorbar()

In [ ]:
imshow_zero_center(h_mat)

A etapa de atualização do método de Newton é mostrada abaixo.

In [ ]:
eps = 1e-3
eye_eps = tf.eye(h_mat.shape[0])*eps

Nota: [Na verdade, não inverta a matriz](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) .

In [ ]:
# X(k+1) = X(k) - (∇²f(X(k)))^-1 @ ∇f(X(k))
# h_mat = ∇²f(X(k))
# g_vec = ∇f(X(k))
update = tf.linalg.solve(h_mat + eye_eps, g_vec)

# Reshape the update and apply it to the variable.
_ = layer1.kernel.assign_sub(tf.reshape(update, layer1.kernel.shape))

Embora isso seja relativamente simples para um único `tf.Variable` , aplicá-lo a um modelo não trivial exigiria concatenação e fatiamento cuidadosos para produzir um Hessiano completo em múltiplas variáveis.

### Lote Jacobiano

Em alguns casos, você deseja considerar o Jacobiano de cada pilha de alvos em relação a uma pilha de fontes, onde os Jacobianos de cada par alvo-fonte são independentes.

Por exemplo, aqui a entrada `x` tem formato `(batch, ins)` e a saída `y` tem formato `(batch, outs)` .


In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = layer2(y)

y.shape

O Jacobiano completo de `y` em relação a `x` tem uma forma de `(batch, ins, batch, outs)` , mesmo se você quiser apenas `(batch, ins, outs)` .

In [ ]:
j = tape.jacobian(y, x)
j.shape

Se os gradientes de cada item na pilha forem independentes, então cada `(batch, batch)` deste tensor é uma matriz diagonal:

In [ ]:
imshow_zero_center(j[:, 0, :, 0])
_ = plt.title('A (batch, batch) slice')

In [ ]:
def plot_as_patches(j):
  # Reorder axes so the diagonals will each form a contiguous patch.
  j = tf.transpose(j, [1, 0, 3, 2])
  # Pad in between each patch.
  lim = tf.reduce_max(abs(j))
  j = tf.pad(j, [[0, 0], [1, 1], [0, 0], [1, 1]],
             constant_values=-lim)
  # Reshape to form a single image.
  s = j.shape
  j = tf.reshape(j, [s[0]*s[1], s[2]*s[3]])
  imshow_zero_center(j, extent=[-0.5, s[2]-0.5, s[0]-0.5, -0.5])

plot_as_patches(j)
_ = plt.title('All (batch, batch) slices are diagonal')

Para obter o resultado desejado, você pode somar a dimensão `batch` duplicado ou então selecionar as diagonais usando `tf.einsum` .


In [ ]:
j_sum = tf.reduce_sum(j, axis=2)
print(j_sum.shape)
j_select = tf.einsum('bxby->bxy', j)
print(j_select.shape)

Seria muito mais eficiente fazer o cálculo sem a dimensão extra em primeiro lugar. O método `GradientTape.batch_jacobian` faz exatamente isso.

In [ ]:
jb = tape.batch_jacobian(y, x)
jb.shape

In [ ]:
error = tf.reduce_max(abs(jb - j_sum))
assert error < 1e-3
print(error.numpy())

Cuidado: `GradientTape.batch_jacobian` verifica apenas se a primeira dimensão da origem e do destino correspondem. Não verifica se os gradientes são realmente independentes. Cabe ao usuário garantir que ele use `batch_jacobian` apenas onde fizer sentido. Por exemplo, adicionar `layers.BatchNormalization` destrói a independência, pois normaliza em toda a dimensão `batch` :

In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
bn = tf.keras.layers.BatchNormalization()
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = bn(y, training=True)
  y = layer2(y)

j = tape.jacobian(y, x)
print(f'j.shape: {j.shape}')

In [ ]:
plot_as_patches(j)

_ = plt.title('These slices are not diagonal')
_ = plt.xlabel("Don't use `batch_jacobian`")

Neste caso, `batch_jacobian` ainda é executado e retorna *algo* com a forma esperada, mas seu conteúdo não tem um significado claro.

In [ ]:
jb = tape.batch_jacobian(y, x)
print(f'jb.shape: {jb.shape}')